<a href="https://colab.research.google.com/github/ptran1203/traffic_sign_detection/blob/main/traffic_signs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## If you need tfrecord data format, you may want to do this step

Add path for [this folder](https://drive.google.com/drive/folders/1msDocIGPo34Xe8JgVYWb0efMzETMTrL6?usp=sharing) to your google drive, It should located at top level of your drive *directory*

In [1]:
from google.colab import drive, output
drive.mount('/content/drive')
# !pip install tf-models-nightly
ZALO_DIR = "/content/drive/My Drive/zalo_ai/"
 
%cd "/content"
!rm -rf traffic_sign_detection
!git clone https://github.com/ptran1203/traffic_sign_detection
%cd traffic_sign_detection
output.clear()

## Copy the file you need for training or inference

In [2]:
!cp "/content/drive/My Drive/zalo_ai/images.tfrecords" /content/traffic_sign_detection/.
# !cp "/content/drive/My Drive/zalo_ai/images_best_preds.tfrecords" /content/traffic_sign_detection/.
!cp "/content/drive/My Drive/zalo_ai/images_test.tfrecords" /content/traffic_sign_detection/.
# !cp "/content/drive/My Drive/zalo_ai/images_private_test.tfrecords" /content/traffic_sign_detection/.
# !cp "/content/drive/MyDrive/zalo_ai/weight_dense.h5" "/content/traffic_sign_detection/weight_dense.h5"
ZALO_DIR = "/content/traffic_sign_detection/"

In [3]:
import json
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
import model as m
import data_processing
import losses
import utils
import math
import datetime
from prediction import Prediction
import os
from collections import Counter
from google.colab.patches import cv2_imshow

DATA_DIR = "/content/drive/My Drive/zalo_ai/za_traffic_2020/za_traffic_2020/"
TFRECORDS_FILE = ZALO_DIR + 'images.tfrecords'
TFRECORDS_FILE_PRED = "/content/drive/MyDrive/zalo_ai/images_best_preds.tfrecords" #ZALO_DIR + 'images_best_preds.tfrecords'
TFRECORDS_FILE_TEST = ZALO_DIR + 'images_test.tfrecords'
WEIGHT_FILE = ZALO_DIR + 'weight_dense.h5'
train_dir = DATA_DIR + "traffic_train/"
test_dir = DATA_DIR + "traffic_public_test/"
json_path = ZALO_DIR + "train_traffic_sign_dataset.json"
label_map = {
    1: "No entry",
    2: "No parking / waiting",
    3: "No turning",
    4: "Max Speed",
    5: "Other prohibition signs",
    6: "Warning",
    7: "Mandatory",
}

WIDTH = 1622
HEIGHT = 626

metadata = json.load(open(json_path, "r"))
test_file_id = "1227,106,7543,7876,7175,6623,877,1169,10568,219,1572,639,12108,10186,9343,11598,377,740,12044,405,6497,12123,451,6725,10990,1062,87,12076,207,7923,232,309,913,7565,10776,11387,9447,1978,11185,9973,9117,10054,8066,8380,10048,8623,6560,1716,9672,12160,1242,12396,512,10060,275,12231,8665,150,10482,255,330,722,6864,10708,374,11104,545,8079,779,7145,6646,542,1739,11141,12334,290,961,10300,11653,501,244,10108,8560,8141,9670,8588,7852,248,10673,8298,1265,632,8546,11878,11260,9472,10333,554,8069,64,749,12290,7854,7903,11163,7898,1478,10397,12407,12504,10908,10431,12464,222,6442,745,10102,421,8547,12071,824,6552,197,482,10122,30,9819,10841,367,9928,878,12302,694,9411,11854,7907,8217,6801,10586,495,420,6350,11876,9093,12257,676,137,12297,12046,8589,882,240,11240,1268,609,12133,531,318,7340,8252,879,9061,414,958,10469,398,677,11381,10954,6846,145,734,12467,7988,725,11966,590,952,10271,52,202,9437,1879,93,10110,407,10437,8320,9044,1985,7653,10943,278,871,12242,9893,118,691,12500,12251,6152,8307,315,83,7856,8865,8569,6545,12184,777,48,479,742,94,6316,6618,9542,11855,7759,7307,7011,7202,114,19,324,801,12299,10249,783,6795,319,8212,7192,9671,12062,12253,103,9290,9701,12121,612,573,419,7329,306,7493,995,791,10981,973,12027,6893,11137,461,288,774,8289,690,11226,7050,12341,9355,863,6901,7849,9965,99,835,9493,7204,490,12243,11617,7844,7206,6165,10425,7156,506,12469,84,7909,10595,11883,154,737,6769,8684,4,12031,6480,964,12379,328,6625,6024,12282,843,1275,1251,7957,105,601,6059,873,9573,481,7542,6607,12069,12462,951,92,813,12455,915,12157,11047,11880,845,122,7512,46,498,9377,6274,53,924,647,502,1212,262,7668,214,11477,191,1839,341,12511,178,668,11450,10396,11177,97,657,345,11,7618,6492,12308,8124,351,10540,0,39,152,970,12202,6764,12204,10411,12132,773,12384,12206,1165,10383,237,1799,6155,1934,827,360,187,11199,23,8531,388,1649,11794,119,11684,6684,9698,10560,11249,12049,1731,12235,12238,11683,9565,7615,916,11771,7182,784,7481,10316,10867,116,470,841,7073,8186,582,6303,12126,10783,9921,866,10426,1013,9230,11827,1936,8337,947,11377,6139,43,603,11786,13,703,8426,714,11373,926,12311,981,9961,10819,12237,12195,11386,9329,10545,12395,10969,11943,11867,1774,1887,968,7786,648,1134,11356,6257,316,1538,11074,806,898,8603,1253,9902,7869,1829,984,6737,12409,12349,8792,8892,75,7484,12312,906,434,6302,10370,10974,12368,923,285,588,8303,249,9264,394,8783,8954,7317,11742,1280,787,12314,8128,907,746,6336,7250,303,199,850,380,11093,6614,6616,708,10013,6606,1488,10859,1100,6346,71,7951,12170,10279,883,1559,7998,6757,550,88,400,331,497,7647,12382,7430,7767,90,8592,1394,100,11209,11513,1050,9862,9165,6343,131,726,1205,6522,760,9282,1337,670,9963,10075,54,815,12322,6766,7127,473,689,174,10064,80,562,6167,10053,936,11752,143,8156,310,658,12410,7236,7633,6304,7819,1690,126,1555,12459,8226,830,127,10814,919,1035,12087,10882,1230,503,423,7060"
test_file_id = sorted([int(c) for c in test_file_id.split(",")])
data_info = [
    {
        "bbox": [[0, 0, 0, 0]],
        "label": [0],
        "id": x,
    } for x in test_file_id
]

if not os.path.isfile(TFRECORDS_FILE):
    raise FileNotFoundError(f"{TFRECORDS_FILE} does not exist")
fdataset = tf.data.TFRecordDataset(TFRECORDS_FILE)

if os.path.isfile(TFRECORDS_FILE_PRED):
    bdataset = tf.data.TFRecordDataset(TFRECORDS_FILE_PRED)
    combined = bdataset.concatenate(fdataset)
else:
    print(f"{TFRECORDS_FILE_PRED} does not exist")
    bdataset = None
    combined = None

/content/drive/MyDrive/zalo_ai/images_best_preds.tfrecords does not exist


In [ ]:
import IPython.display as display
from data_processing import *
autotune = tf.data.experimental.AUTOTUNE

num_classes= 7
batch_size= 2

data_processor =data_processing.DataProcessing(400, 154)

label_encoder = m.LabelEncoder()
dataset = (combined or fdataset).map(data_processor.preprocess_data)
dataset = dataset.shuffle(8 * batch_size)
dataset = dataset.padded_batch(
    batch_size,
    padding_values=(0.0, 1e-8, tf.cast(-1, tf.int64)),
    drop_remainder=True,
)
dataset = dataset.map(
    label_encoder.encode_batch, num_parallel_calls=autotune
)
dataset = dataset.apply(tf.data.experimental.ignore_errors())
dataset = dataset.prefetch(autotune)

val_size = 0
train_size = 4500 # + 1211 HARDCODE if using some test image (pseudo labeling)
train_steps_per_epoch = train_size // batch_size
train_steps = 4 * 10000
epochs = train_steps // train_steps_per_epoch

learning_rates = [2.5e-05, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)
optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDrive/zalo_ai/weight_dense.h5",
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

backbone = "densenet121"
WEIGHT_FILE = ZALO_DIR + "weight_dense.h5"
label_smoothing = False

model = m.RetinaNet(num_classes, backbone=backbone)
model.compile(optimizer=optimizer, 
              loss=losses.RetinaNetLoss(num_classes, label_smoothing=label_smoothing))

# Train 1 sample first so we can load the weight
model.fit(dataset.take(1))
utils.try_ignore_error(model.load_weights, WEIGHT_FILE)

H = model.fit(dataset.repeat(),
            epochs=epochs,
            steps_per_epoch=train_steps_per_epoch,
            callbacks=callbacks_list)

## Get the best predictions and do pseudo labeling

In [ ]:
preds = json.load(open("/content/drive/MyDrive/zalo_ai/submission.json"))
preds
best_preds = []
for pred in preds:
    if pred["score"] > 0.95:
        best_preds.append(pred)

all_cates = [b["category_id"] for b in best_preds]
id_list = [b["image_id"] for b in best_preds]
best_preds_data = {}

for pred in best_preds:
    id_ = pred["image_id"]
    bbox = [int(b) for b in pred["bbox"]]
    if id_ in best_preds_data:
        best_preds_data[id_]["bbox"].append(bbox)
        best_preds_data[id_]["label"].append(pred["category_id"])
    else:
        best_preds_data[id_] = {
            "id": id_,
            "bbox": [bbox],
            "label": [pred["category_id"]]
        }

best_preds_data = list(best_preds_data.values())
best_preds_data

In [ ]:
# bdataset = tf.data.TFRecordDataset(TFRECORDS_FILE_PRED)
# btfdataset = bdataset.map(DataProcessing(convert_xywh=0).preprocess_data)
for img, bbox, label in btfdataset.skip(100).take(5):
    utils.visualize_detections(img ,bbox, label, label,
                               (15,15))


## Inference model

In [ ]:
import time
image = tf.keras.Input(shape=[None, None, 3], name="image")
weight_path = "/content/drive/MyDrive/zalo_ai/weight_dense.h5"
model.fit(np.random.rand(1, 896, 2304, 3), np.random.rand(1, 386694, 5))
model.load_weights(weight_path)
predictions = model(image, training=False)
detections = m.DecodePredictions(confidence_threshold=0.6,
                                 num_classes=num_classes,
                                 max_detections_per_class=10,
                                 nms_iou_threshold=0.35,
                                 verbose=0)(image, predictions)
inference_model = tf.keras.Model(inputs=image, outputs=detections)
ptest_dataset = tf.data.TFRecordDataset("images_private_test.tfrecords")
time.ctime(os.path.getmtime(weight_path))

In [ ]:
with open("private_id_list", "r") as f:
    ptest_id_list = f.read().split(",")
    ptest_id_list = [int(x) for x in ptest_id_list]

data_info = [
        {
            "bbox": [[0, 0, 0, 0]],
            "label": [0],
            "id": int(x),
        } for x in test_file_id
    ]


In [ ]:
from prediction import Prediction as P
import datetime

test_dataset = tf.data.TFRecordDataset("images_test.tfrecords")

class Prediction(P):
    pass

skip = 123#ptest_id_list.index(134)
take = 1
predictor = Prediction(inference_model, 230, HEIGHT)
# print(ptest_id_list[skip:skip+take])
for sample in test_dataset.skip(skip).take(take):
    start = datetime.datetime.now()
    image, boxes, scores, classes = predictor.detect_single_image(sample, show=False)
    print("Detect in {} - {} objects".format(
        datetime.datetime.now() - start,
        len(boxes)
    ))
    class_names = [
        label_map[int(x)] for x in classes
    ]
    utils.visualize_detections(
        image,
        boxes,
        class_names,
        scores,
        figsize=(15,15),
        color=[1,1,1]
    )


In [ ]:
submission = []
idx = 0
import datetime
start = datetime.datetime.now()
for sample in test_dataset:
    image, boxes, scores, classes = predictor.detect_single_image(sample)
    if not isinstance(boxes, list):
        boxes = boxes.numpy()
        scores = scores.numpy()
        classes = classes.numpy()

    for i in range(len(boxes)):
        box = boxes[i]
        x1, y1, x2, y2 = box
        xywh = [x1, y1, x2 - x1, y2 - y1]
        # print(xywh, data_info[idx]["id"])
        score = scores[i]
        cls = classes[i]
        submission.append({
            "image_id": data_info[idx]["id"],
            "category_id": int(cls),
            "bbox": [float(z) for z in xywh],
            "score": float(score),
        })

    utils._print_progress("{}/{}".format(idx, 585))
    idx += 1

print("Predict in {}".format(datetime.datetime.now() - start))
with open("submission.json", "w") as f:
    json.dump(submission, f, indent=2)